# ReAct con LangChain (Avanzado)

## Definición
ReAct combina razonamiento + acción: el agente decide, ejecuta herramientas y usa observaciones para el siguiente paso.

## Cuándo usarlo
- Cuando necesitas herramientas reales (auditoría, búsqueda, validación).
- Cuando la trazabilidad de pasos es requisito de negocio.

## Cuándo NO usarlo
- Si no hay tools que agreguen valor.
- Si una llamada directa ya cumple calidad/costo/latencia.

## Flujo conceptual

```mermaid
graph TD
A[State] --> B[Decide Action]
B --> C[ANALIZAR_PERFIL]
C --> D[GENERAR_MENSAJE]
D --> E[AUDITAR_RESPETO]
E --> F[FINAL_ANSWER]
```

In [1]:
# Setup de notebook
from __future__ import annotations

import importlib.util
import inspect
import json
import sys
from pathlib import Path
from IPython.display import Markdown, display


def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "pyproject.toml").exists():
            return p
    raise RuntimeError("No se encontro raiz del repo")


def load_module(path: Path, module_name: str):
    spec = importlib.util.spec_from_file_location(module_name, path)
    module = importlib.util.module_from_spec(spec)
    if spec is None or spec.loader is None:
        raise RuntimeError(f"No se pudo cargar modulo: {path}")
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

ROOT = find_repo_root(Path.cwd())
print('Repo root:', ROOT)

Repo root: /Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry


In [2]:
script = ROOT / '03_langchain_prompting/ReAct_LangChain/Notebooks/01_react_langchain_avanzado.py'
module = load_module(script, 'react_langchain_nb')
run_fn = module.run_react_langchain
print('Script:', script)
print('Firma:', inspect.signature(run_fn))

Script: /Users/carlosdaniel/Documents/Projects/labor_projects/Henry/2026/01-introduction_ai_engineering/ai_engineering_henry/03_langchain_prompting/ReAct_LangChain/Notebooks/01_react_langchain_avanzado.py
Firma: (profile: 'dict | None' = None, verbose: 'bool' = True) -> 'dict'


## Código fuente principal (visible)

In [3]:
print(inspect.getsource(run_fn)[:7000])

def run_react_langchain(profile: dict | None = None, verbose: bool = True) -> dict:
    root = find_repo_root(Path.cwd())
    load_dotenv(root / ".env")

    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError("OPENAI_API_KEY no esta definida en .env")

    model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
    build_context_packet = load_context_builder(root)
    llm = ChatOpenAI(model=model, temperature=0.4, api_key=api_key)

    if profile is None:
        profile = {
            "tipo_persona": "arquitecta apasionada por fotografia urbana",
            "gustos": ["cafes tranquilos", "jazz", "viajes cortos", "cafes tranquilos"],
            "estilo": "intelectual pero relajado",
            "contexto": "match reciente, primera interaccion",
        }

    context_packet = build_context_packet(
        profile=profile,
        module="03_langchain_prompting",
        strategy="react_langchain",
    )

    analizar_tool = StructuredTool.from_function(
  

## Ejemplo 1: caso base

In [4]:
result_1 = run_fn(verbose=False)
print('Modelo:', result_1['__model'])
print('Context hash:', result_1['__context_hash'])
print('Final answer:')
print(json.dumps(result_1['final_answer'], ensure_ascii=False, indent=2))
print('Trace preview:')
print(json.dumps(result_1['trace_preview'], ensure_ascii=False, indent=2))

Modelo: gpt-4o-mini
Context hash: 12995565f4f0
Final answer:
{
  "opener": "Vi que te mueves en arquitecta apasionada por fotografia urbana, que tema te entusiasma tanto que podrias hablar horas sin aburrirte?",
  "follow_up": "Si te parece, yo comparto uno y comparamos notas.",
  "why_it_works": [
    "Conecta con identidad e intereses.",
    "Mantiene tono curioso y respetuoso."
  ],
  "trace_summary": [
    "Analizar el perfil de la persona para entender sus intereses.",
    "Generar un mensaje atractivo acorde a sus intereses y estilo.",
    "Auditar el mensaje para asegurar que respeta el contexto y los intereses de la persona."
  ]
}
Trace preview:
[
  {
    "action": "ANALIZAR_PERFIL",
    "observation_excerpt": "{\"persona\": \"arquitecta apasionada por fotografia urbana\", \"estilo_preferido\": \"intelectual pero relajado\", \"insights\": [\"Intereses detectad"
  },
  {
    "action": "GENERAR_MENSAJE",
    "observation_excerpt": "{\"opener\": \"Vi que te mueves en arquitecta a

In [5]:
display(Markdown('## Flujo real usado en ejecución'))
display(Markdown('```mermaid\n' + result_1['__flow_mermaid'] + '\n```'))

## Flujo real usado en ejecución

```mermaid
graph TD
A[State] --> B[Decide Action]
B --> C[ANALIZAR_PERFIL]
C --> D[GENERAR_MENSAJE]
D --> E[AUDITAR_RESPETO]
E --> F[FINAL_ANSWER]
```

## Ejemplo 2: coqueteo (asistente latino experto en enamorar)

In [6]:
profile_coqueteo = {
  "tipo_persona": "asistente latino experto en enamorar con elegancia",
  "gustos": [
    "salsa romantica",
    "cafes bohemios",
    "poesia urbana",
    "paseos nocturnos"
  ],
  "estilo": "carismatico, coqueto, respetuoso, humor picante sutil",
  "contexto": "quiere iniciar una conversacion de coqueteo sin frases prefabricadas"
}
result_2 = run_fn(profile=profile_coqueteo, verbose=False)
print('Context hash coqueteo:', result_2['__context_hash'])
print('Final answer coqueteo:')
print(json.dumps(result_2['final_answer'], ensure_ascii=False, indent=2))

Context hash coqueteo: 6f918a97a9d4
Final answer coqueteo:
{
  "opener": "Vi que te mueves en asistente latino experto en enamorar con elegancia, que tema te entusiasma tanto que podrias hablar horas sin aburrirte?",
  "follow_up": "Si te parece, yo comparto uno y comparamos notas.",
  "why_it_works": [
    "Conecta con identidad e intereses.",
    "Mantiene tono curioso y respetuoso."
  ],
  "trace_summary": [
    "Analizar el perfil del agente para entender su contexto y gustos.",
    "Generar un mensaje de coqueteo carismático y auténtico.",
    "Auditar el respeto en el mensaje para asegurar que cumple con el estilo deseado."
  ]
}


## Errores típicos en producción
- Tools con contratos ambiguos: el agente se vuelve impredecible.
- Sin guardrails de secuencia: deriva de acciones y debugging costoso.
- Sin auditoría de respeto: riesgo reputacional en outputs sensibles.
- Sobrecargar de herramientas irrelevantes: más latencia, menos precisión.

## Autocrítica
- ReAct aporta control operativo, pero cuesta más que CoT simple.
- La calidad sigue dependiendo del contexto; herramientas malas no rescatan un mal diseño.